## 初期値

In [1]:
import os

os.chdir("/work")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process
from logic.manage.utils.excel_tools import create_label_rows_generic, sort_by_cell_row

# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_receive = "/work/data/output/debug_receive.parquet"
debug_shipping = "/work/data/output/debug_shipping.parquet"
debug_yard = "/work/data/output/debug_yard.parquet"

dfs = {
    "receive": pd.read_parquet(debug_receive),
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]

2025-04-28 14:07:43,783 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ factory_report.py の process 関数を登録しました
2025-04-28 14:07:43,837 [WARNING] (__init__.py:26) [fb6959af9847/root] ❌ 処理関数の読み込みエラー: balance_sheet → ImportError: cannot import name 'process_factory_report' from 'logic.manage.processors.balance_sheet.balance_sheet_fact' (/work/logic/manage/processors/balance_sheet/balance_sheet_fact.py)
2025-04-28 14:07:43,851 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ average_sheet.py の process 関数を登録しました
2025-04-28 14:07:43,861 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ management_sheet.py の process 関数を登録しました


### Process

In [ ]:
# 処理の統合
def process(dfs: dict) -> pd.DataFrame:
    logger = app_logger()
    """
    Streamlitの選択に基づき、工場日報（処分パート）を処理するエントリーポイント関数。
    """

    logger = app_logger()

    # --- テンプレート設定の取得 ---
    template_key = "balance_sheet"
    template_config = get_template_config()[template_key]
    template_name = template_config["key"]
    csv_keys = template_config["required_files"]
    logger.info(f"[テンプレート設定読込] key={template_key}, files={csv_keys}")

    # --- CSVの読み込み ---
    df_dict = load_all_filtered_dataframes(dfs, csv_keys, template_name)
    df_receive = df_dict.get("receive")
    df_shipping = df_dict.get("shipping")
    df_yard = df_dict.get("yard")

    # --- 個別処理 ---
    logger.info("▶️ 搬出量データ処理開始")
    master_csv_facotry = process_factory_report(df_shipping)

    logger.info("▶️ 処分費データ処理開始")
    master_csv_facotry = process_factory_report(df_shipping)

### 工場日報から搬出量

In [8]:
def process_factory_report(dfs):
    logger = app_logger()
    from logic.manage.factory_report import process

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["factory"]
    master_csv = load_master_and_template(master_path)

    # 工場日報からdfを読込
    df_factory = process(dfs)

    # 搬出量を抜出
    after_master_csv = reflect_total_from_factory(master_csv, df_factory)

    return after_master_csv


master_csv_facotry = process_factory_report(dfs)

2025-04-28 14:09:05,290 [INFO] (factory_report.py:27) [fb6959af9847/root] [テンプレート設定読込] key=factory_report, files=['yard', 'shipping']
2025-04-28 14:09:05,318 [INFO] (factory_report.py:35) [fb6959af9847/root] ▶️ 出荷処分データ処理開始
/work/utils/data_cleaning.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['8339' '5238' '8340' '8085' '6421' '8337' '8215' '8336' '8327' '8327'
 '8327' '8327' '705' '1185' '8076' '8077' '5199' '8334' '6797' '8338'
 '8341' '8080' '349' '6786' '5227' '8329' '8360' '8363' '8398' '8679'
 '339' '8082']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[valid, col] = df.loc[valid, col].apply(lambda x: str(int(float(x))))
2025-04-28 14:09:05,371 [INFO] (factory_report_shobun.py:40) [fb6959af9847/root] ✅ 出荷処分の帳票生成が完了しました。
2025-04-28 14:09:05,372 [INFO] (factory_report.py:38) [fb6959af9847/root] ▶️ 出荷有価データ処理開始
2025-04-28 14:09:05,407 [INFO] (su

In [ ]:
from utils.value_setter import set_value_fast


def reflect_total_from_factory(master_csv, df_factory):
    total_sum = df_factory.loc[df_factory["大項目"] == "総合計", "値"].squeeze()

    match_columns = ["大項目"]
    match_value = ["搬出量"]
    set_value_fast(master_csv, match_columns, match_value, total_sum)

    return master_csv

### 仕入処分費

In [ ]:
def process_syobun_cost():
    return